In [1]:
import pandas as pd

In [37]:
# now that we have the df for twitter sentiment of each day fron 20170101
sentiment_filename = 'dailysentiments.csv'
stock_filename = 'stockprice_raw.csv'
sentiments = pd.read_csv(sentiment_filename, engine='python',index_col='date')
stocks = pd.read_csv(stock_filename, engine = 'python',index_col='Date')
#sentiments = sentiments.drop(columns=sentiments.columns[0])

In [38]:
sentiments

,pos_mean_likes,pos_mean_replies,pos_mean_retweet,neu_mean_likes,neu_mean_replies,neu_mean_retweet,neg_mean_likes,neg_mean_replies,neg_mean_retweet
date,,,,,,,,,
20170101,4.207272,3.828412,3.232838,8.028677,8.366872,9.094131,0.264051,0.304716,0.173031
20170102,6.322558,6.260311,5.908047,6.780715,7.166920,7.121932,1.182442,0.858484,1.255735
20170103,1.233669,1.469026,1.436088,4.423724,4.048554,4.352946,4.342607,4.482420,4.210966
20170104,0.821791,0.382300,0.921218,4.941648,2.870712,4.963726,0.486562,2.996988,0.365056
20170105,2.220672,2.516691,2.811572,6.303775,5.975591,5.727157,0.566462,0.598628,0.552180
...,...,...,...,...,...,...,...,...,...
20201227,0.886511,0.862483,0.781388,0.093095,0.110687,0.179754,0.020394,0.026830,0.038858
20201228,0.055150,0.117387,0.041629,0.310698,0.787195,0.255934,0.634153,0.095418,0.702437
20201229,0.471613,0.463504,0.395673,0.441821,0.497922,0.437942,0.086566,0.038573,0.166385


In [39]:
# set the size of the sliding window of stock price prediction
window_size = 1
# use Open_prev to find the opening price of the previous day
stocks['Open_prev'] = stocks['Open'].shift(window_size)

In [7]:
from datetime import timedelta
dates = stocks.index
dates = pd.to_datetime(dates)

#to correct for if the gap is more than one day
for k in range(stocks.shape[0]):
  if k > 0 and (dates[k] - dates[k-1]) > timedelta(days =1):
    idx = stocks.index[k]
    stocks.at[idx, 'Open_prev'] = stocks.iloc[k-1]['Open']

In [8]:
stocks

,date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Open_prev
Date,,,,,,,,,
2016-12-30,20161230,43.259998,43.500000,42.335999,42.737999,23213000,0,0.0,NaN
2017-01-03,20170103,42.972000,44.066002,42.192001,43.397999,29616500,0,0.0,43.259998
2017-01-04,20170104,42.950001,45.599998,42.862000,45.397999,56067500,0,0.0,42.972000
2017-01-05,20170105,45.284000,45.495998,44.389999,45.349998,29558500,0,0.0,42.950001
2017-01-06,20170106,45.386002,46.062000,45.090000,45.801998,27639500,0,0.0,45.284000
...,...,...,...,...,...,...,...,...,...
2020-12-28,20201228,674.510010,681.400024,660.799988,663.690002,32278600,0,0.0,642.989990
2020-12-29,20201229,661.000000,669.900024,655.000000,665.989990,22910800,0,0.0,674.510010
2020-12-30,20201230,672.000000,696.599976,668.359985,694.780029,42846000,0,0.0,661.000000


In [14]:
#add in features of previous two days' stock prices
stocks['prev2_Diff'] = stocks.shift(2)["Close"] - stocks.shift(3)["Open"]
stocks

,date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Open_prev,Diff,prev2_Diff
Date,,,,,,,,,,,
2016-12-30,20161230,43.259998,43.500000,42.335999,42.737999,23213000,0,0.0,NaN,NaN,NaN
2017-01-03,20170103,42.972000,44.066002,42.192001,43.397999,29616500,0,0.0,43.259998,1.0,NaN
2017-01-04,20170104,42.950001,45.599998,42.862000,45.397999,56067500,0,0.0,42.972000,1.0,NaN
2017-01-05,20170105,45.284000,45.495998,44.389999,45.349998,29558500,0,0.0,42.950001,1.0,0.138000
2017-01-06,20170106,45.386002,46.062000,45.090000,45.801998,27639500,0,0.0,45.284000,1.0,2.425999
...,...,...,...,...,...,...,...,...,...,...,...
2020-12-28,20201228,674.510010,681.400024,660.799988,663.690002,32278600,0,0.0,642.989990,1.0,-2.020020
2020-12-29,20201229,661.000000,669.900024,655.000000,665.989990,22910800,0,0.0,674.510010,-1.0,29.570007
2020-12-30,20201230,672.000000,696.599976,668.359985,694.780029,42846000,0,0.0,661.000000,1.0,20.700012


In [15]:
stocks['Diff'] = stocks['Close'] - stocks['Open_prev']
stocks.loc[stocks['Diff']> 0, 'Diff'] = 1
stocks.loc[stocks['Diff']< 0, 'Diff'] = -1
stocks.loc[stocks['Diff'] == 0, 'Diff'] = 0
stock_label_df = stocks[['date','prev2_Diff','Diff']]

In [16]:
stock_label_df
#stock_label_df.to_csv('stockprice_label.csv')

,date,prev2_Diff,Diff
Date,,,
2016-12-30,20161230,NaN,NaN
2017-01-03,20170103,NaN,1.0
2017-01-04,20170104,NaN,1.0
2017-01-05,20170105,0.138000,1.0
2017-01-06,20170106,2.425999,1.0
...,...,...,...
2020-12-28,20201228,-2.020020,1.0
2020-12-29,20201229,29.570007,-1.0
2020-12-30,20201230,20.700012,1.0


In [47]:
'''
previous code:
# Create a df with sentiment analysis from previous two days given the dateindex 
sentiment_prev2days = pd.concat([sentiments.shift(1),sentiments.shift(2)],axis=1)
sentiment_prev2days = sentiment_prev2days.dropna()
sentiment_prev2days.insert(0,'date', sentiment_prev2days.index)
sentiment_prev2days.index.names = ['Date']
sentiment_prev2days
# note that 1448 tweet days vs. 1007 valid stock labels

'''

#sentiment analysis window
sentiment_window_size = 7

features = ["pos_mean_likes", "pos_mean_replies", "pos_mean_retweet","neu_mean_likes", "neu_mean_replies", "neu_mean_retweet","neg_mean_likes", "neg_mean_replies", "neg_mean_retweet"]
day_features = [("day{}_"+s).format(sentiment_window_size) for s in features]
#first shift to the day before stock price window
sentiment_shift = sentiments.shift(window_size+1)
sentiment_shift.columns = day_features

k = sentiment_window_size - 1

#accumulate features of sentiment analysis window
for i in range(3, sentiment_window_size + 2):
  day_features = [("day{}_"+s).format(k) for s in features]
  df = sentiments.shift(i)
  df.columns = day_features
  sentiment_shift = pd.concat([sentiment_shift,df],axis=1)
  k -= 1

# #accumulate features of sentiment analysis window
# for i in range(3, sentiment_window_size + 2):
#   sentiment_shift = pd.concat([sentiment_shift,sentiments.shift(i)],axis=1)

sentiment_shift = sentiment_shift.dropna()

sentiment_shift

,day7_pos_mean_likes,day7_pos_mean_replies,day7_pos_mean_retweet,day7_neu_mean_likes,day7_neu_mean_replies,day7_neu_mean_retweet,day7_neg_mean_likes,day7_neg_mean_replies,day7_neg_mean_retweet,day6_pos_mean_likes,...,day2_neg_mean_retweet,day1_pos_mean_likes,day1_pos_mean_replies,day1_pos_mean_retweet,day1_neu_mean_likes,day1_neu_mean_replies,day1_neu_mean_retweet,day1_neg_mean_likes,day1_neg_mean_replies,day1_neg_mean_retweet
date,,,,,,,,,,,,,,,,,,,,,
20170109,0.584927,0.552184,0.598106,2.431541,2.477603,2.434837,0.209339,0.196020,0.192863,2.263765,...,1.255735,4.207272,3.828412,3.232838,8.028677,8.366872,9.094131,0.264051,0.304716,0.173031
20170110,4.102322,2.973540,3.970256,4.156664,5.277351,4.300867,0.074348,0.082443,0.062210,0.584927,...,4.210966,6.322558,6.260311,5.908047,6.780715,7.166920,7.121932,1.182442,0.858484,1.255735
20170111,2.783762,1.733815,4.389617,6.720076,3.673627,5.217909,4.781877,8.878273,4.678189,4.102322,...,0.365056,1.233669,1.469026,1.436088,4.423724,4.048554,4.352946,4.342607,4.482420,4.210966
20170112,3.540877,2.980973,3.052879,2.571007,2.997780,3.021332,0.138117,0.271247,0.175789,2.783762,...,0.552180,0.821791,0.382300,0.921218,4.941648,2.870712,4.963726,0.486562,2.996988,0.365056
20170113,2.688410,2.520085,2.844737,5.373914,5.619566,5.162004,0.271010,0.193683,0.326593,3.540877,...,0.216844,2.220672,2.516691,2.811572,6.303775,5.975591,5.727157,0.566462,0.598628,0.552180
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201227,0.233273,0.461738,0.186157,0.448576,0.485939,0.455544,0.318151,0.052323,0.358299,0.387610,...,0.091325,0.317752,0.297707,0.382962,0.397160,0.551848,0.416572,0.285088,0.150445,0.200466
20201228,0.316511,0.409077,0.364218,0.397077,0.424900,0.539451,0.286412,0.166023,0.096330,0.233273,...,0.085256,0.403248,0.267194,0.323698,0.482170,0.550139,0.584976,0.114582,0.182667,0.091325
20201229,0.886511,0.862483,0.781388,0.093095,0.110687,0.179754,0.020394,0.026830,0.038858,0.316511,...,0.508401,0.399282,0.383977,0.334963,0.367910,0.351789,0.420118,0.073144,0.104569,0.085256


In [18]:
dataframe = sentiment_shift.merge(stock_label_df,how='left', left_on='date', right_on='date')
dataframe = dataframe.dropna()
dataframe

,date,pos_mean_likes,pos_mean_replies,pos_mean_retweet,neu_mean_likes,neu_mean_replies,neu_mean_retweet,neg_mean_likes,neg_mean_replies,neg_mean_retweet,...,pos_mean_replies,pos_mean_retweet,neu_mean_likes,neu_mean_replies,neu_mean_retweet,neg_mean_likes,neg_mean_replies,neg_mean_retweet,prev2_Diff,Diff
0,20170109,0.584927,0.552184,0.598106,2.431541,2.477603,2.434837,0.209339,0.196020,0.192863,...,3.828412,3.232838,8.028677,8.366872,9.094131,0.264051,0.304716,0.173031,2.399998,1.0
1,20170110,4.102322,2.973540,3.970256,4.156664,5.277351,4.300867,0.074348,0.082443,0.062210,...,6.260311,5.908047,6.780715,7.166920,7.121932,1.182442,0.858484,1.255735,0.517998,1.0
2,20170111,2.783762,1.733815,4.389617,6.720076,3.673627,5.217909,4.781877,8.878273,4.678189,...,1.469026,1.436088,4.423724,4.048554,4.352946,4.342607,4.482420,4.210966,0.869999,-1.0
3,20170112,3.540877,2.980973,3.052879,2.571007,2.997780,3.021332,0.138117,0.271247,0.175789,...,0.382300,0.921218,4.941648,2.870712,4.963726,0.486562,2.996988,0.365056,0.180000,1.0
4,20170113,2.688410,2.520085,2.844737,5.373914,5.619566,5.162004,0.271010,0.193683,0.326593,...,2.516691,2.811572,6.303775,5.975591,5.727157,0.566462,0.598628,0.552180,-0.454002,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1419,20201224,0.072582,0.075541,0.060729,0.404326,0.423167,0.430870,0.523092,0.501292,0.508401,...,0.232098,0.134903,0.634783,0.643418,0.779769,0.117921,0.124484,0.085328,-25.899963,1.0
1423,20201228,0.316511,0.409077,0.364218,0.397077,0.424900,0.539451,0.286412,0.166023,0.096330,...,0.267194,0.323698,0.482170,0.550139,0.584976,0.114582,0.182667,0.091325,-2.020020,1.0
1424,20201229,0.886511,0.862483,0.781388,0.093095,0.110687,0.179754,0.020394,0.026830,0.038858,...,0.383977,0.334963,0.367910,0.351789,0.420118,0.073144,0.104569,0.085256,29.570007,-1.0
1425,20201230,0.055150,0.117387,0.041629,0.310698,0.787195,0.255934,0.634153,0.095418,0.702437,...,0.075541,0.060729,0.404326,0.423167,0.430870,0.523092,0.501292,0.508401,20.700012,1.0


In [19]:

filename = "database_prev{}days.csv".format(sentiment_window_size)
filename

'database_prev7days.csv'

In [20]:
dataframe.to_csv(filename)